### Introduction

The database that will be created will comprise data pulled from the following two sources

data source 1: https://www.transtats.bts.gov/ot_delay/OT_DelayCause1.asp?20=E <br>
dictionary: https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr<br>

data source 2: https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import requests
import json
from sqlalchemy import create_engine



### Load  data

In [2]:
columns = ['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay']

flights_raw = pd.read_csv('Raw data/Airline_Delay_Cause.csv', usecols=columns)
flights = flights_raw.copy(deep=True)

### Understanding the data

In [3]:
flights.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2022,6,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",133.0,14.0,3.0,0.0,853.0,625.0,2.0,56.0,0.0,170.0
1,2022,6,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",89.0,15.0,1.0,1.0,853.0,545.0,48.0,43.0,0.0,217.0
2,2022,6,9E,Endeavor Air Inc.,ACK,"Nantucket, MA: Nantucket Memorial",120.0,12.0,5.0,1.0,2045.0,424.0,901.0,237.0,0.0,483.0
3,2022,6,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",78.0,15.0,1.0,0.0,1647.0,626.0,231.0,182.0,0.0,608.0
4,2022,6,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",176.0,22.0,3.0,0.0,1114.0,701.0,92.0,166.0,0.0,155.0


In [4]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319750 entries, 0 to 319749
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   year                 319750 non-null  int64  
 1   month                319750 non-null  int64  
 2   carrier              319750 non-null  object 
 3   carrier_name         319750 non-null  object 
 4   airport              319750 non-null  object 
 5   airport_name         319750 non-null  object 
 6   arr_flights          319262 non-null  float64
 7   arr_del15            319022 non-null  float64
 8   arr_cancelled        319262 non-null  float64
 9   arr_diverted         319262 non-null  float64
 10  arr_delay            319262 non-null  float64
 11  carrier_delay        319262 non-null  float64
 12  weather_delay        319262 non-null  float64
 13  nas_delay            319262 non-null  float64
 14  security_delay       319262 non-null  float64
 15  late_aircraft_del

In [5]:
flights.isnull().sum()

year                     0
month                    0
carrier                  0
carrier_name             0
airport                  0
airport_name             0
arr_flights            488
arr_del15              728
arr_cancelled          488
arr_diverted           488
arr_delay              488
carrier_delay          488
weather_delay          488
nas_delay              488
security_delay         488
late_aircraft_delay    488
dtype: int64

In [6]:
def add_airport(airport:str) -> str:
    if 'airport' not in airport.lower():
        airport = airport + ' Airport'
    return airport

flights['airport_name'] = flights['airport_name'].apply(lambda x: add_airport(x))
flights.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2022,6,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",133.0,14.0,3.0,0.0,853.0,625.0,2.0,56.0,0.0,170.0
1,2022,6,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional Airport",89.0,15.0,1.0,1.0,853.0,545.0,48.0,43.0,0.0,217.0
2,2022,6,9E,Endeavor Air Inc.,ACK,"Nantucket, MA: Nantucket Memorial Airport",120.0,12.0,5.0,1.0,2045.0,424.0,901.0,237.0,0.0,483.0
3,2022,6,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International Airport",78.0,15.0,1.0,0.0,1647.0,626.0,231.0,182.0,0.0,608.0
4,2022,6,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field Ai...",176.0,22.0,3.0,0.0,1114.0,701.0,92.0,166.0,0.0,155.0


In [7]:
rename_columns = {
    'arr_flights': 'num_flights',
    'arr_del15': 'num_delayed',
    'arr_cancelled': 'num_cancelled',
    'arr_diverted': 'num_diverted',
    'arr_delay': 'total_delay'
}
flights = flights.rename(columns=rename_columns)

### Normalize table

#### Create separate table for airports 

In [8]:
# find unique airports and store them in a separate data frame
airports_unique = flights['airport_name'].unique()
airports = flights.drop_duplicates('airport', keep='first')[['airport', 'airport_name']].reset_index(drop=True)

def extract_loc(airport:str) -> list:
    "returns a list of airport, city, and state for each given string"
    airport_name = airport.split(': ')[1]
    city_state = airport.split(':')[0]
    city, state = city_state.split(', ')
    return [airport_name, city, state]

flights['airport_name'] = flights['airport_name'].apply(lambda x: x.split(': ')[1])

In [9]:
airports.head()

,airport,airport_name
0,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ..."
1,ABY,"Albany, GA: Southwest Georgia Regional Airport"
2,ACK,"Nantucket, MA: Nantucket Memorial Airport"
3,AEX,"Alexandria, LA: Alexandria International Airport"
4,AGS,"Augusta, GA: Augusta Regional at Bush Field Ai..."


In [10]:
airports_data = [extract_loc(airport=airport) for airport in airports['airport_name']]
airports = airports.drop('airport_name', axis=1)

location = pd.DataFrame(airports_data, columns=['airport_name', 'city', 'state'])
airports = pd.concat([airports, location], axis=1, ignore_index=True)
airports = airports.reset_index()
airports['index'] = airports['index']+1
airports.columns = ['airport_id', 'airport', 'airport_name', 'city', 'state']
airports.head()

,airport_id,airport,airport_name,city,state
0,1,ABE,Lehigh Valley International Airport,Allentown/Bethlehem/Easton,PA
1,2,ABY,Southwest Georgia Regional Airport,Albany,GA
2,3,ACK,Nantucket Memorial Airport,Nantucket,MA
3,4,AEX,Alexandria International Airport,Alexandria,LA
4,5,AGS,Augusta Regional at Bush Field Airport,Augusta,GA


####  Create a foreign key in the airport_delays table to link to the airports table

In [11]:
# create airport_id column in the airport_delays column (foreign key)
airport_id_map = dict(zip(airports['airport'], airports['airport_id']))
flights['airport_id'] = flights['airport'].map(airport_id_map)
flights = flights.drop(['airport', 'airport_name'], axis=1)

In [12]:
flights.shape

(319750, 15)

#### Create separate table for carrier data

In [13]:
carrier = flights.drop_duplicates(['carrier', 'carrier_name'])[['carrier', 'carrier_name']].reset_index(drop=True)
dup = carrier[carrier['carrier'].duplicated()]['carrier']
carrier[carrier['carrier'].isin(list(dup))].sort_values('carrier')

,carrier,carrier_name
0,9E,Endeavor Air Inc.
23,9E,Pinnacle Airlines Inc.
32,DH,Independence Air
34,DH,Atlantic Coast Airlines
17,EV,ExpressJet Airlines LLC
18,EV,ExpressJet Airlines Inc.
24,EV,Atlantic Southeast Airlines
8,MQ,Envoy Air
22,MQ,American Eagle Airlines Inc.
10,OH,PSA Airlines Inc.


<b>Changes to address duplicates:</b>

9E - Replace 'Pinnacle Airlines Inc.' with 'Endeavor Air Inc.'.<br>
source: https://en.wikipedia.org/wiki/Pinnacle_Airlines_Corp.

DH - Replace 'Atlantic Coast Airlines' with 'Independence Air'<br>
source: https://en.wikipedia.org/wiki/Atlantic_Coast_Airlines

EV - replace 'ExpressJet Airlines LLC' with 'ExpressJet Airlines Inc.'<br>
replace 'Atlantic Southeast Airlines' with 'ExpressJet Airlines Inc.'<br>
source: https://en.wikipedia.org/wiki/Atlantic_Southeast_Airlines

MQ - Replace 'American Eagle Airlines Inc.' with 'Envoy Air'<br>
source: https://en.wikipedia.org/wiki/American_Eagle_(airline_brand)

OH - for Comair Inc., replace OH with OH(1)<br>
source: https://www.bts.gov/topics/airlines-and-airports/airline-codes

In [14]:
replace_carrier_name = {
    'Pinnacle Airlines Inc.': 'Endeavor Air Inc.',
    'Atlantic Coast Airlines': 'Independence Air',
    'ExpressJet Airlines LLC': 'ExpressJet Airlines Inc.',
     'Atlantic Southeast Airlines': 'ExpressJet Airlines Inc.',
    'American Eagle Airlines Inc.':  'Envoy Air',
}

flights['carrier_name'] = flights['carrier_name'].astype('string').replace(replace_carrier_name)

flights.loc[flights['carrier_name']=='Comair Inc.', 'carrier'] = 'OH (1)'

In [15]:
# test for duplicates again
carrier = flights.drop_duplicates(['carrier', 'carrier_name'])[['carrier', 'carrier_name']].reset_index(drop=True)
carrier[carrier['carrier'].duplicated()]

,carrier,carrier_name


In [16]:
# create new data frame with carrier info
carrier = flights.drop_duplicates(['carrier', 'carrier_name'])[['carrier', 'carrier_name']].reset_index(drop=True)
carrier = carrier.reset_index()
carrier['index'] = carrier['index'] + 1
carrier = carrier.rename(columns={'index': 'carrier_id'})
carrier.head()

,carrier_id,carrier,carrier_name
0,1,9E,Endeavor Air Inc.
1,2,AA,American Airlines Inc.
2,3,AS,Alaska Airlines Inc.
3,4,B6,JetBlue Airways
4,5,DL,Delta Air Lines Inc.


Dictionary

arr_flights --> total operations
<br>arr_del15 --> Arrival Delay Indicator, 15 Minutes or More (1=Yes)
<br>arr_cancelled -->  count of cancelled flights
<br>arr_diverted --> count of diverted flights
<br>carrier_delay --> Carrier Delay, in Minutes
<br>weather_delay --> 	Weather Delay, in Minutes
<br>nas_delay --> National Air System Delay, in Minutes
<br>security_delay --> 	Security Delay, in Minutes
<br>late_aircraft_delay --> Late Aircraft Delay, in Minutes

#### Create foreign key in arrival_delays table to connect to the carrier table

In [17]:
# create a foreign key for the carrier table
carrier_map = dict(zip(carrier['carrier'], carrier['carrier_id']))
flights['carrier_id'] = flights['carrier'].map(carrier_map)
flights = flights.drop(['carrier', 'carrier_name'], axis=1)

In [18]:
flights.head()

,year,month,num_flights,num_delayed,num_cancelled,num_diverted,total_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,airport_id,carrier_id
0,2022,6,133.0,14.0,3.0,0.0,853.0,625.0,2.0,56.0,0.0,170.0,1,1
1,2022,6,89.0,15.0,1.0,1.0,853.0,545.0,48.0,43.0,0.0,217.0,2,1
2,2022,6,120.0,12.0,5.0,1.0,2045.0,424.0,901.0,237.0,0.0,483.0,3,1
3,2022,6,78.0,15.0,1.0,0.0,1647.0,626.0,231.0,182.0,0.0,608.0,4,1
4,2022,6,176.0,22.0,3.0,0.0,1114.0,701.0,92.0,166.0,0.0,155.0,5,1


#### Create primary key for airport_delays table 

In [19]:
flights = flights.sort_values(['airport_id', 'carrier_id', 'year', 'month']).reset_index(drop=True)
flights = flights.reset_index()
flights['index'] = flights['index']+1
flights = flights.rename(columns={'index': 'row_id'})

In [20]:
order = ['row_id', 'airport_id', 'carrier_id', 'year', 'month', 'num_flights', 'num_delayed', 'num_cancelled', 'num_diverted', 'total_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']
flights = flights[order]
flights.head()

,row_id,airport_id,carrier_id,year,month,num_flights,num_delayed,num_cancelled,num_diverted,total_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,1,1,1,2007,1,93.0,25.0,5.0,0.0,1176.0,509.0,68.0,374.0,0.0,225.0
1,2,1,1,2007,2,80.0,28.0,6.0,0.0,1210.0,533.0,40.0,482.0,0.0,155.0
2,3,1,1,2007,3,80.0,20.0,2.0,0.0,811.0,477.0,0.0,219.0,0.0,115.0
3,4,1,1,2007,4,81.0,16.0,1.0,0.0,422.0,105.0,0.0,277.0,0.0,40.0
4,5,1,1,2007,5,84.0,18.0,1.0,0.0,844.0,249.0,0.0,325.0,0.0,270.0


#### Impute missing values 

In [21]:
dict(flights.isnull().sum())

{'row_id': 0,
 'airport_id': 0,
 'carrier_id': 0,
 'year': 0,
 'month': 0,
 'num_flights': 488,
 'num_delayed': 728,
 'num_cancelled': 488,
 'num_diverted': 488,
 'total_delay': 488,
 'carrier_delay': 488,
 'weather_delay': 488,
 'nas_delay': 488,
 'security_delay': 488,
 'late_aircraft_delay': 488}

In [22]:
flights = flights.fillna(method='bfill', axis='rows')

In [23]:
dict(flights.isnull().sum())

{'row_id': 0,
 'airport_id': 0,
 'carrier_id': 0,
 'year': 0,
 'month': 0,
 'num_flights': 0,
 'num_delayed': 0,
 'num_cancelled': 0,
 'num_diverted': 0,
 'total_delay': 0,
 'carrier_delay': 0,
 'weather_delay': 0,
 'nas_delay': 0,
 'security_delay': 0,
 'late_aircraft_delay': 0}

In [24]:
# convert counts to int variable
int_columns = ['num_flights', 'num_delayed', 'num_cancelled', 'num_diverted']
for col in int_columns:
    flights[col] = flights[col].astype('int')

### Pull data from the second data source

In [25]:
# source (large and medium hub only): https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States

url = 'https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States'
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

# collect data from first table
table1 = soup.find('tbody')
data = []
for row in table1.find_all('tr'):
    row = [val.text.rstrip('\n') for val in row if val.text.rstrip('\n')]
    data.append(row)

# store large hub table into a data frame
large_hub = pd.DataFrame(data)
large_hub.columns = large_hub.iloc[0,:]
large_hub = large_hub.iloc[1:,:]
large_hub['airport_type'] = ['Large Hub' for _ in range(len(large_hub))]
large_hub = large_hub[['IATACode', 'airport_type']]
large_hub.columns = ['airport_code', 'airport_type']

# collect data from second table
table2 = soup.find_all('table')[1]
data = []
for row in table2.find_all('tr'):
    row = [val.text.rstrip('\n') for val in row if val.text.rstrip('\n')]
    data.append(row)

# store medium hub table into a data frame
medium_hub = pd.DataFrame(data)
medium_hub.columns = medium_hub.iloc[0,:]
medium_hub = medium_hub.iloc[1:,:]
medium_hub['airport_type'] = ['Medium Hub' for _ in range(len(medium_hub))]
medium_hub = medium_hub[['IATACode', 'airport_type']]
medium_hub.columns = ['airport_code', 'airport_type']

# merge two tables 
airport_type = pd.concat([large_hub, medium_hub]).reset_index(drop=True)
airport_type.head()

,airport_code,airport_type
0,ATL,Large Hub
1,DFW,Large Hub
2,DEN,Large Hub
3,ORD,Large Hub
4,LAX,Large Hub


### Export data frames as .csv files

In [26]:
flights.to_csv('Processed data/Flights Data.csv', index=False)
airports.to_csv('Processed data/Airport Data.csv', index=False)
carrier.to_csv('Processed data/Carrier Data.csv', index=False)
airport_type.to_csv('Processed data/Airport Type.csv', index=False)